In [1]:
%run config.py

In [2]:
import numpy as np
import pandas as pd
from copy import deepcopy

#visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

# NLP modules we will use for text normalization
import re #regex 
import nltk # the natural language toolkit
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag

# feature construction
from sklearn.feature_extraction.text import TfidfVectorizer #use this to create BoW matrix

In [3]:
df = pd.read_csv('Data/customer_support_tickets.csv')

In [4]:
import pyLDAvis.lda_model


#modeling and dimensionality reduction for visuaization
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE

In [5]:
df['Ticket Description'] = df.apply(lambda x: x['Ticket Description'].replace("{product_purchased}", x['Product Purchased']), axis=1)


In [6]:
df['Ticket Description']

0       I'm having an issue with the GoPro Hero. Pleas...
1       I'm having an issue with the LG Smart TV. Plea...
2       I'm facing a problem with my Dell XPS. The Del...
3       I'm having an issue with the Microsoft Office....
4       I'm having an issue with the Autodesk AutoCAD....
                              ...                        
8464    My LG OLED is making strange noises and not fu...
8465    I'm having an issue with the Bose SoundLink Sp...
8466    I'm having an issue with the GoPro Action Came...
8467    I'm having an issue with the PlayStation. Plea...
8468    There seems to be a hardware problem with my P...
Name: Ticket Description, Length: 8469, dtype: object

In [7]:
df['Ticket Description'] = df['Ticket Description'].str.lower()

In [8]:
cleaned_df = df[['Ticket ID','Product Purchased','Ticket Type','Ticket Description']]

In [9]:
cleaned_df

,Ticket ID,Product Purchased,Ticket Type,Ticket Description
0,1,GoPro Hero,Technical issue,i'm having an issue with the gopro hero. pleas...
1,2,LG Smart TV,Technical issue,i'm having an issue with the lg smart tv. plea...
2,3,Dell XPS,Technical issue,i'm facing a problem with my dell xps. the del...
3,4,Microsoft Office,Billing inquiry,i'm having an issue with the microsoft office....
4,5,Autodesk AutoCAD,Billing inquiry,i'm having an issue with the autodesk autocad....
...,...,...,...,...
8464,8465,LG OLED,Product inquiry,my lg oled is making strange noises and not fu...
8465,8466,Bose SoundLink Speaker,Technical issue,i'm having an issue with the bose soundlink sp...
8466,8467,GoPro Action Camera,Technical issue,i'm having an issue with the gopro action came...
8467,8468,PlayStation,Product inquiry,i'm having an issue with the playstation. plea...


In [10]:
from stop_words import get_stop_words

def process_ticket(ticket_text, min_length):
    
    # get common stop words that we'll remove during tokenization/text normalization
    stop_words = list(get_stop_words('en'))         #About 900 stopwords
    nltk_words = list(stopwords.words('english')) #About 150 stopwords
    stop_words.extend(nltk_words)

    #initialize lemmatizer
    wnl = WordNetLemmatizer()

    # helper function to change nltk's part of speech tagging to a wordnet format.
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None
   

    # lower case everything
    ticket_lower = ticket_text.lower()

    ticket_lower = re.sub(r"@[a-z0-9_]+|#[a-z0-9_]+|http\S+", "", ticket_lower).strip().replace("\r", "").replace("\n", "").replace("\t", "")

    # remove stop words and punctuations 
    
    ticket_norm = [x for x in word_tokenize(ticket_lower) if ((x.isalpha()) & (x not in stop_words)) ]

    #  POS detection on the result will be important in telling Wordnet's lemmatizer how to lemmatize
    
    # creates list of tuples with tokens and POS tags in wordnet format
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(ticket_norm))) 

    # now we are going to have a cutoff here. any tokenized cocument with length < min length will be removed from corpus
    if len(wordnet_tagged) <= min_length:
        return ''
    else:
         # rejoins lemmatized sentence 
        ticket_norm = " ".join([wnl.lemmatize(x[0], x[1]) for x in wordnet_tagged if x[1] is not None])
        return ticket_norm

In [11]:
cleaned_df['Ticket Description Cleaned'] = cleaned_df['Ticket Description'].apply(process_ticket, args = [10])


/var/folders/_x/3hvhfs550_j07qx5c00fkgnc0000gn/T/ipykernel_76789/1391231502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Ticket Description Cleaned'] = cleaned_df['Ticket Description'].apply(process_ticket, args = [10])


In [12]:
# canon_df = cleaned_df[cleaned_df['Product Purchased']=='Canon EOS']

In [13]:
# for ticket_type in canon_df['Ticket Type'].unique():
#     corpus = canon_df[canon_df['Ticket Type']==ticket_type]['Ticket Description Cleaned']
#     print(ticket_type)
#     print('-------')
#     vectorizer = TfidfVectorizer()
#     X_train = vectorizer.fit_transform(corpus)
#     X_train

#     topic_model = NMF(n_components = 5)
#     topic_model.fit(X_train)

#     # to get H
#     H = topic_model.transform(X_train) # transform document into topic vector representation

#     # to get W 
#     W = topic_model.components_ # word component weights for each topic

#     for index,topic in enumerate(W):
#         print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
#         print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-25:]])
#         print('\n')

In [14]:
# Check if 'Assigned Topic' and 'Topic Words' columns exist, if not, create them
if 'Assigned Topic' not in cleaned_df.columns:
    cleaned_df['Assigned Topic'] = None
if 'Topic Words' not in cleaned_df.columns:
    cleaned_df['Topic Words'] = None

# Initialize a dictionary to store top words for each topic of each product and ticket type
topic_words_dict = {}

# Iterate over each product
for product in cleaned_df['Product Purchased'].unique():
    # Filter for tickets related to the current product
    product_df = cleaned_df[cleaned_df['Product Purchased'] == product]
    
    for ticket_type in product_df['Ticket Type'].unique():
        # Selecting the subset of the DataFrame for the current ticket type
        subset_df = product_df[product_df['Ticket Type'] == ticket_type]
        corpus = subset_df['Ticket Description']
        ticket_ids = subset_df.index  # Using DataFrame index as a proxy for ticket ID if 'Ticket ID' column does not exist

        # Initialize TF-IDF Vectorizer and NMF Model
        vectorizer = TfidfVectorizer()
        X_train = vectorizer.fit_transform(corpus)
        topic_model = NMF(n_components=5, random_state=42)  # Added random_state for reproducibility
        topic_model.fit(X_train)

        # Transform document into topic vector representation
        H = topic_model.transform(X_train)  # Document-topic matrix
        W = topic_model.components_  # Topic-term matrix

        # For each topic, store the top words
        for topic_idx, topic in enumerate(W):
            top_words = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-20:]]  # Get top 10 words
            topic_key = f"{product}_{ticket_type}_Topic{topic_idx}"
            topic_words_dict[topic_key] = ', '.join(top_words)  # Store as a comma-separated string

        # Assign tickets to topics based on the highest topic weight
        for ticket_index, topic_vector in zip(ticket_ids, H):
            # Find the topic with the highest weight for this ticket
            assigned_topic = topic_vector.argmax()
            topic_key = f"{product}_{ticket_type}_Topic{assigned_topic}"
            # Update the 'Assigned Topic' in cleaned_df directly
            cleaned_df.at[ticket_index, 'Assigned Topic'] = topic_key

# Map the 'Assigned Topic' to its corresponding top words
cleaned_df['Topic Words'] = cleaned_df['Assigned Topic'].map(topic_words_dict)


/var/folders/_x/3hvhfs550_j07qx5c00fkgnc0000gn/T/ipykernel_76789/488352925.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Assigned Topic'] = None
/var/folders/_x/3hvhfs550_j07qx5c00fkgnc0000gn/T/ipykernel_76789/488352925.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Topic Words'] = None
/var/folders/_x/3hvhfs550_j07qx5c00fkgnc0000gn/T/ipykernel_76789/488352925.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [15]:
cleaned_df[cleaned_df['Product Purchased']=='Canon EOS']['Topic Words']

19      please, product, ensure, safe, security, conce...
36      an, with, eos, canon, could, you, me, option, ...
85      factory, performed, password, is, can, canon, ...
144     working, turning, now, yesterday, until, not, ...
199     could, update, that, started, and, please, can...
                              ...                        
8222    using, my, on, settings, and, configurations, ...
8229    peripherals, adapters, cables, ve, different, ...
8324    please, product, ensure, safe, security, conce...
8341    would, other, are, the, devices, fine, doesn, ...
8349    assist, right, canon, eos, steps, you, in, me,...
Name: Topic Words, Length: 240, dtype: object

In [16]:
import random
index = random.randint(0, cleaned_df.shape[0])

# cleaned_df.iloc[[index]]

product = cleaned_df.iloc[index]['Product Purchased']
issue_type = cleaned_df.iloc[index]['Ticket Type']
topic_words = cleaned_df.iloc[index]['Topic Words']
description = cleaned_df.iloc[index]['Ticket Description']

print(product)
print(issue_type)
print(topic_words)
print()
print(description)


Amazon Echo
Technical issue
you, kai, thank, issue, bug, amazon, echo, of, related, to, recently, firmware, happening, afterward, updated, could, be, started, update, the

i'm having an issue with the amazon echo. please assist. i'll fix it as quickly as i can! thank you."

it's not just the text that changes these numbers that makes the results appear to be wrong i've recently updated the firmware of my amazon echo, and the issue started happening afterward. could it be related to the update?


In [17]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key = api_key)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a customer support bot. Your job is to give helpful advice when a customer writes in with a given issue. We have used a machine learning model to assign a topic to their specific request based on the top 20 words associated with that topic. The response should always contain a resolution, there will be no opportunity for followup from the user."},
    {"role": "user", "content": f"Product is {product}, Issue type is {issue_type}, Topic words are {topic_words}"}
  ]
)

print(response.choices[0].message.content)

I'm sorry to hear that you're experiencing technical issues with your Amazon Echo following a recent firmware update. To resolve this issue, I recommend performing a factory reset on your Amazon Echo. This can help clear out any bugs or glitches that may have occurred due to the update. Here's how you can do it:

1. Press and hold the "Action" button on your Amazon Echo for about 25 seconds until the light ring turns orange. 
2. Once the light ring turns blue, Alexa will greet you. Follow the instructions to complete the setup process again.

Performing a factory reset should help resolve any issues you're facing with your Amazon Echo post the firmware update.


# GloVe Embedding

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_vectors = vectorizer.fit_transform(cleaned_df['Ticket Description'])


In [25]:
def load_glove_embeddings(path):
    embeddings_dict = {}
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

# Example path, adjust as needed
glove_path = 'Data/glove.6B.100d.txt'
glove_embeddings = load_glove_embeddings(glove_path)


In [39]:
def document_to_glove_vector(text, glove_embeddings):
    word_vectors = []
    # Check if text is None and treat it as an empty string
    text = text if text is not None else ""
    for word in text.split():
        if word in glove_embeddings:
            word_vectors.append(glove_embeddings[word])
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        # Return a zero vector if the document is empty
        return np.zeros(100)  # 100-dimensional GloVe vectors

# Apply this function to each document
glove_vectors = np.array([document_to_glove_vector(doc, glove_embeddings) for doc in cleaned_df['Ticket Description']])

In [40]:
# # For TF-IDF vectors
# tsne_model_tfidf = TSNE(n_components=2, random_state=42)
# tsne_tfidf = tsne_model_tfidf.fit_transform(tfidf_vectors.toarray())  # Convert sparse matrix to dense

# For GloVe vectors
tsne_model_glove = TSNE(n_components=2, random_state=42)
tsne_glove = tsne_model_glove.fit_transform(glove_vectors)

AttributeError: 'NoneType' object has no attribute 'split'